In [1]:
import hashlib
import hmac
import json
import os
import time
from datetime import datetime

import pandas as pd
import requests
from dotenv import load_dotenv

load_dotenv()

True

In [28]:
api_key = os.getenv("BYBIT_API_KEY")
secret_key = os.getenv("BYBIT_SECRET_KEY")

httpClient = requests.Session()

# maximum allowable time difference (in milliseconds) between the server time and the request time
# this parameter helps ensure that the request is not rejected due to time discrepancies
recv_window = str(10000)
url = "https://api.bybit.com"


# Please make sure that the timestamp parameter adheres to the following rule:
# server_time - recv_window <= timestamp < server_time + 1000

def HTTP_Request(endpoint, method, payload):
    time_stamp = str(int(time.time() * 10 ** 3))
    signature = hmac.new(bytes(secret_key, "utf-8"), f"{time_stamp}{api_key}{recv_window}{payload}".encode("utf-8"),
                         hashlib.sha256).hexdigest()
    response = httpClient.request(method, url + endpoint + "?" + payload, headers={
        'X-BAPI-API-KEY': api_key,
        'X-BAPI-SIGN': signature,
        'X-BAPI-SIGN-TYPE': '2',
        'X-BAPI-TIMESTAMP': time_stamp,
        'X-BAPI-RECV-WINDOW': recv_window,
        'Content-Type': 'application/json'
    })
    return response.text


start_date = int(datetime.strptime("2024-01-01", "%Y-%m-%d").timestamp())
end_date = int(datetime.strptime("2024-12-11", "%Y-%m-%d").timestamp())

response = HTTP_Request(endpoint="/v5/market/kline", method="GET",
                        payload=f"category=spot&symbol=ETHUSDT&interval=60&start={start_date}000&end={end_date}000&limit=1000")

In [29]:
# > list[0]: startTime	string	Start time of the candle (ms)
# > list[1]: openPrice	string	Open price
# > list[2]: highPrice	string	Highest price
# > list[3]: lowPrice	string	Lowest price
# > list[4]: closePrice	string	Close price. Is the last traded price when the candle is not closed
# > list[5]: volume	string	Trade volume. Unit of contract: pieces of contract. Unit of spot: quantity of coins
# > list[6]: turnover	string	Turnover. Unit of figure: quantity of quota coin
df = pd.DataFrame(json.loads(response)["result"]["list"]).set_axis(
    ["startTime", "openPrice", "highPrice", "lowPrice", "closePrice", "volume", "turnover"], axis=1)
# .sort_values(
#     "startTime"))
# Convert startTime from unix format to format like "2024-12-01 00:00:00"
# df["startTime"] = pd.to_datetime(df["startTime"].astype(int), unit="ms")


In [30]:
df

,startTime,openPrice,highPrice,lowPrice,closePrice,volume,turnover
0,1733792400000,3767.34,3774.75,3724.89,3755.39,13563.71429,50863651.4696175
1,1733788800000,3711.75,3781.07,3677.89,3767.34,21912.11167,81946877.2012598
2,1733785200000,3713.69,3734,3688.56,3711.75,19729.39931,73263505.0470212
3,1733781600000,3697.66,3731.07,3507.45,3713.69,43835.98448,160774332.1540192
4,1733778000000,3743.15,3747.6,3453.99,3697.66,83772.42195,306273586.5110691
...,...,...,...,...,...,...,...
984,1730250000000,2620.67,2627.78,2599.39,2625.69,8560.45665,22390956.1807933
985,1730246400000,2638.81,2641.88,2615.67,2620.67,5546.31948,14581343.032647
986,1730242800000,2636.76,2640.19,2630.62,2638.81,5516.15075,14536614.6803741
987,1730239200000,2622.63,2648.78,2620.71,2636.76,6605.76398,17409353.7629482


In [51]:
import pandas as pd
from typing import List, Tuple, Dict


class FibonacciRetracement:
    def __init__(self):
        """Initialize Fibonacci retracement levels and data structures"""
        self.fib_levels = [0, 0.236, 0.382, 0.5, 0.618, 0.786, 1]
        self.df = None

    def load_data(self, data: List[Dict]) -> None:
        """
        Load and preprocess kline data

        Parameters:
            data: List of dictionaries containing OHLCV data with columns:
                  startTime, openPrice, highPrice, lowPrice, closePrice, volume, turnover
        """
        self.df = pd.DataFrame(data)

        # Convert string columns to numeric
        numeric_columns = ['openPrice', 'highPrice', 'lowPrice', 'closePrice', 'volume', 'turnover']
        for col in numeric_columns:
            self.df[col] = pd.to_numeric(self.df[col], errors='coerce')

        # Convert timestamp to datetime
        self.df['startTime'] = pd.to_datetime(self.df['startTime'].astype(float), unit='ms')
        self.df.set_index('startTime', inplace=True)

    def calculate_fibonacci_levels(self, high: float, low: float) -> Dict[float, float]:
        """
        Calculate Fibonacci retracement levels

        Parameters:
            high: Highest price point
            low: Lowest price point

        Returns:
            Dictionary of Fibonacci levels and their corresponding prices
        """
        diff = high - low
        return {level: high - (diff * level) for level in self.fib_levels}

    def find_swing_points(self, window: int = 20) -> Tuple[List[int], List[int]]:
        """
        Find swing high and low points in the price data

        Parameters:
            window: Number of candles to look before and after for swing point confirmation

        Returns:
            Tuple of lists containing indices of swing highs and lows
        """
        if self.df is None:
            raise ValueError("Data not loaded. Call load_data first.")

        highs = []
        lows = []

        for i in range(window, len(self.df) - window):
            # Check for swing high
            if self.df['highPrice'].iloc[i] == max(self.df['highPrice'].iloc[i - window:i + window + 1]):
                highs.append(i)
            # Check for swing low
            if self.df['lowPrice'].iloc[i] == min(self.df['lowPrice'].iloc[i - window:i + window + 1]):
                lows.append(i)

        return highs, lows

    def identify_trend(self, periods: int = 20) -> str:
        """
        Identify current market trend using SMA

        Parameters:
            periods: Number of periods for moving average calculation

        Returns:
            String indicating trend ('UPTREND' or 'DOWNTREND')
        """
        if self.df is None:
            raise ValueError("Data not loaded. Call load_data first.")

        sma = self.df['closePrice'].rolling(window=periods).mean()
        current_price = float(self.df['closePrice'].iloc[-1])
        current_sma = float(sma.iloc[-1])

        return 'UPTREND' if current_price > current_sma else 'DOWNTREND'

    def find_trading_signals(self, window: int = 20) -> pd.DataFrame:
        """
        Find potential trading signals based on Fibonacci retracements

        Parameters:
            window: Number of candles for swing point detection

        Returns:
            DataFrame with signals and Fibonacci levels
        """
        if self.df is None:
            raise ValueError("Data not loaded. Call load_data first.")

        # Create a copy and fill with WAIT signals
        df_signals = self.df.copy()
        df_signals['Signal'] = 'WAIT'
        df_signals['Fib_Level'] = None

        # Find swing points
        highs, lows = self.find_swing_points(window)
        print(f"Found {len(highs)} highs and {len(lows)} lows")

        # Need at least one high and one low
        if len(highs) == 0 or len(lows) == 0:
            print("Not enough swing points found")
            return df_signals

        # Get trend and recent high/low
        trend = self.identify_trend()
        recent_low = float(self.df['lowPrice'].iloc[lows[-1]])
        recent_high = float(self.df['highPrice'].iloc[highs[-1]])

        # Calculate Fibonacci levels
        fib_levels = self.calculate_fibonacci_levels(recent_high, recent_low)

        # Check each price point for signals
        for i in range(len(df_signals)):
            price = float(df_signals['closePrice'].iloc[i])

            for level, fib_price in fib_levels.items():
                if abs(price - fib_price) / fib_price < 0.01:  # Within 1% of Fibonacci level
                    if trend == 'UPTREND':
                        df_signals.at[df_signals.index[i], 'Signal'] = 'BUY'
                        df_signals.at[df_signals.index[i], 'Fib_Level'] = level
                    elif trend == 'DOWNTREND':
                        df_signals.at[df_signals.index[i], 'Signal'] = 'SELL'
                        df_signals.at[df_signals.index[i], 'Fib_Level'] = level

        return df_signals

    def backtest_strategy(self, initial_capital: float = 10000.0) -> pd.DataFrame:
        """
        Backtest the Fibonacci retracement strategy

        Parameters:
            initial_capital: Starting capital for the backtest

        Returns:
            DataFrame with backtest results including positions and portfolio values
        """
        if self.df is None:
            raise ValueError("Data not loaded. Call load_data first.")

        df_backtest = self.find_trading_signals()
        df_backtest['Position'] = 'NONE'
        df_backtest['Portfolio_Value'] = initial_capital

        position = 'NONE'
        entry_price = 0.0
        stop_loss = 0.0
        take_profit = 0.0

        for i in range(1, len(df_backtest)):
            current_signal = df_backtest['Signal'].iloc[i]
            df_backtest.at[df_backtest.index[i], 'Position'] = position

            if position == 'NONE':
                if current_signal == 'BUY':
                    position = 'LONG'
                    entry_price = float(df_backtest['closePrice'].iloc[i])
                    stop_loss = entry_price * 0.95
                    take_profit = entry_price * 1.1
                    df_backtest.at[df_backtest.index[i], 'Position'] = 'LONG'

                elif current_signal == 'SELL':
                    position = 'SHORT'
                    entry_price = float(df_backtest['closePrice'].iloc[i])
                    stop_loss = entry_price * 1.05
                    take_profit = entry_price * 0.9
                    df_backtest.at[df_backtest.index[i], 'Position'] = 'SHORT'

            elif position == 'LONG':
                current_low = float(df_backtest['lowPrice'].iloc[i])
                current_high = float(df_backtest['highPrice'].iloc[i])

                if current_low < stop_loss:
                    position = 'NONE'
                    df_backtest.at[df_backtest.index[i], 'Position'] = 'STOP_LOSS'
                elif current_high > take_profit:
                    position = 'NONE'
                    df_backtest.at[df_backtest.index[i], 'Position'] = 'TAKE_PROFIT'

            elif position == 'SHORT':
                current_low = float(df_backtest['lowPrice'].iloc[i])
                current_high = float(df_backtest['highPrice'].iloc[i])

                if current_high > stop_loss:
                    position = 'NONE'
                    df_backtest.at[df_backtest.index[i], 'Position'] = 'STOP_LOSS'
                elif current_low < take_profit:
                    position = 'NONE'
                    df_backtest.at[df_backtest.index[i], 'Position'] = 'TAKE_PROFIT'

            # Update portfolio value
            current_close = float(df_backtest['closePrice'].iloc[i])
            last_value = df_backtest['Portfolio_Value'].iloc[i - 1]

            if position == 'NONE':
                df_backtest.at[df_backtest.index[i], 'Portfolio_Value'] = last_value
            elif position == 'LONG':
                pnl = (current_close - entry_price) / entry_price
                df_backtest.at[df_backtest.index[i], 'Portfolio_Value'] = last_value * (1 + pnl)
            elif position == 'SHORT':
                pnl = (entry_price - current_close) / entry_price
                df_backtest.at[df_backtest.index[i], 'Portfolio_Value'] = last_value * (1 + pnl)

        return df_backtest

    def get_current_signal(self) -> Dict:
        """
        Get the current trading signal and additional information

        Returns:
            Dictionary containing current signal, trend, price, and other relevant information
        """
        if self.df is None or len(self.df) == 0:
            raise ValueError("No data available")

        signals = self.find_trading_signals()
        current_signal = signals.iloc[-1]

        return {
            'signal': current_signal['Signal'],
            'fib_level': current_signal['Fib_Level'],
            'trend': self.identify_trend(),
            'current_price': float(self.df['closePrice'].iloc[-1]),
            'timestamp': self.df.index[-1]
        }

    def get_performance_metrics(self, backtest_results: pd.DataFrame) -> Dict:
        """
        Calculate performance metrics from backtest results

        Parameters:
            backtest_results: DataFrame containing backtest results

        Returns:
            Dictionary of performance metrics
        """
        initial_value = float(backtest_results['Portfolio_Value'].iloc[0])
        final_value = float(backtest_results['Portfolio_Value'].iloc[-1])
        total_return = (final_value - initial_value) / initial_value * 100

        trades = backtest_results[backtest_results['Position'].isin(['STOP_LOSS', 'TAKE_PROFIT'])]
        winning_trades = trades[trades['Position'] == 'TAKE_PROFIT']

        metrics = {
            'Total Return (%)': round(total_return, 2),
            'Number of Trades': len(trades),
            'Win Rate (%)': round(len(winning_trades) / len(trades) * 100 if len(trades) > 0 else 0, 2),
            'Average Win (%)': 10.0,  # Fixed due to take profit setting
            'Average Loss (%)': -5.0,  # Fixed due to stop loss setting
            'Final Portfolio Value': round(final_value, 2)
        }

        return metrics

In [52]:
fib = FibonacciRetracement()
fib.load_data(df)

In [53]:
signals = fib.find_trading_signals(window=6)

Found 48 highs and 50 lows


In [54]:
signals.query("Signal == 'BUY' or Signal == 'SELL'")

,openPrice,highPrice,lowPrice,closePrice,volume,turnover,Signal,Fib_Level
startTime,,,,,,,,
2024-11-07 00:00:00,2721.90,2725.72,2699.36,2725.72,6085.16457,1.649656e+07,SELL,0.236
2024-11-06 23:00:00,2735.60,2741.94,2711.30,2721.90,8958.28599,2.442618e+07,SELL,0.236
2024-11-06 22:00:00,2687.09,2745.17,2684.11,2735.60,10191.34207,2.765571e+07,SELL,0
2024-11-06 21:00:00,2689.76,2703.90,2682.49,2687.09,7949.58821,2.139506e+07,SELL,0.618
2024-11-06 20:00:00,2687.10,2701.82,2669.17,2689.76,11821.62230,3.173402e+07,SELL,0.618
2024-11-06 19:00:00,2677.94,2687.49,2658.40,2687.10,9426.63542,2.518286e+07,SELL,0.618
2024-11-06 18:00:00,2652.30,2679.09,2650.88,2677.94,8334.54742,2.222913e+07,SELL,0.786
2024-11-06 17:00:00,2656.16,2660.86,2643.00,2652.30,7803.64503,2.070248e+07,SELL,1
2024-11-06 16:00:00,2633.90,2665.39,2633.22,2656.16,9305.13394,2.467323e+07,SELL,1


In [55]:
results = fib.backtest_strategy()

Found 17 highs and 14 lows


In [58]:
results

,openPrice,highPrice,lowPrice,closePrice,volume,turnover,Signal,Fib_Level,Position,Portfolio_Value
startTime,,,,,,,,,,
2024-12-10 01:00:00,3767.34,3774.75,3724.89,3755.39,13563.71429,5.086365e+07,WAIT,None,NONE,1.000000e+04
2024-12-10 00:00:00,3711.75,3781.07,3677.89,3767.34,21912.11167,8.194688e+07,WAIT,None,NONE,1.000000e+04
2024-12-09 23:00:00,3713.69,3734.00,3688.56,3711.75,19729.39931,7.326351e+07,WAIT,None,NONE,1.000000e+04
2024-12-09 22:00:00,3697.66,3731.07,3507.45,3713.69,43835.98448,1.607743e+08,WAIT,None,NONE,1.000000e+04
2024-12-09 21:00:00,3743.15,3747.60,3453.99,3697.66,83772.42195,3.062736e+08,WAIT,None,NONE,1.000000e+04
...,...,...,...,...,...,...,...,...,...,...
2024-10-30 01:00:00,2620.67,2627.78,2599.39,2625.69,8560.45665,2.239096e+07,WAIT,None,SHORT,3.200139e+06
2024-10-30 00:00:00,2638.81,2641.88,2615.67,2620.67,5546.31948,1.458134e+07,WAIT,None,SHORT,3.181779e+06
2024-10-29 23:00:00,2636.76,2640.19,2630.62,2638.81,5516.15075,1.453661e+07,WAIT,None,SHORT,3.141373e+06


In [59]:
metrics = fib.get_performance_metrics(results)

In [60]:
metrics

{'Total Return (%)': 30738.09,
 'Number of Trades': 0,
 'Win Rate (%)': 0,
 'Average Win (%)': 10.0,
 'Average Loss (%)': -5.0,
 'Final Portfolio Value': 3083809.24}

In [ ]:
Ò